In [2]:
from typing import Annotated

from dotenv import load_dotenv

load_dotenv()
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph , START,END
from langgraph.graph.message import  add_messages

In [5]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list,add_messages]

def chatbot(state:State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node",chatbot)

builder.add_edge(START,"chatbot_node")
builder.add_edge("chatbot_node",END)

graph = builder.compile()



In [7]:
message = {"role":"user","content":"which team won 2024 t20 world cup?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[AIMessage(content='Australia won the 2024 T20 World Cup.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--1594f63f-7627-41be-a398-7a3099edcea5-0', usage_metadata={'input_tokens': 14, 'output_tokens': 15, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})]

In [9]:
state= None
while True:
    in_message = input("You:")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State ={
            "messages": [{"role":"user","content":in_message}]
        }
    else:
        state["messages"].append({"role":"user","content":in_message})

    state = graph.invoke(state)
    print("Bot:",state["messages"][-1].content)

Bot: Neil Armstrong was the first person to walk on the Moon.
